## <center> RecSys. Home Assignment 1</center>

Один из важных навыков для построения рекомендательных систем - это умение корректно считать метрики качества ранжирования.

В этой домашке вам нужно потренироваться в этом и имплементировать метрики Precision@k, Recall@k, MNAP@k и NDCG@k по формулам, чтобы дальше переиспользовать при построении своих рекомендательных моделей. 

Критерии оценивания:
* Что-то пытался сделать, дописал свой код, но ничего не получилось - 1 балл. 
* Не совсем корректная имплементация одной из 4 метрик, прохождение части тестов - 1 балл. 
* Корректная имплементация одной из 4 метрик, прохождение всех тестов - 2 балла. 
* +1 балл, если получится написать Precision@k, Recall@k без циклов.
* +1 балл, если получится написать NDCG@k, MNAP@k без циклов.

Дедлайн сдачи - **24 октября 23:59**. 

Формат сдачи - отправить Jupyter notebook на почту ananyeva.me@gmail.com с темой письма "[RecSys HW1]" и названием файла [Name_Surname]_HW1.ipynb.  

Удачи!

In [52]:
from typing import NamedTuple, Union
import tests
import torch

In [53]:
class PrepareTargetResult(NamedTuple):
    values: torch.Tensor
    indices: torch.Tensor


def validate_metric_inputs(output: torch.Tensor, target: torch.Tensor) -> None:
    if output.size() != target.size():
        raise IndexError(
            "Unequal sizes for output and target: "
            f"output - {output.size()}, target - {target.size()}."
        )
    if not (target.eq(0) | target.eq(1)).all():
        raise ValueError(
            "Target contains values outside of 0 and 1." f"\nTarget:\n{target}"
        )


def prepare_target(
    output: torch.Tensor, target: torch.Tensor, return_indices: bool = False
) -> Union[torch.Tensor, PrepareTargetResult]:
    validate_metric_inputs(output, target)
    # Define order by sorted output scores.
    indices = output.argsort(dim=-1, descending=True)
    sorted_target = torch.gather(target, index=indices, dim=-1)
    return (
        PrepareTargetResult(sorted_target, indices) if return_indices else sorted_target
    )


def nan_to_num(tensor: torch.Tensor, nan: float = 0.0) -> torch.Tensor:
    return torch.where(
        torch.isnan(tensor) | torch.isinf(tensor),
        torch.full_like(tensor, fill_value=nan),
        tensor,
    )

# Precision

$$P@k = \frac{1}{k} \sum_{i=1}^k [y_{i} = 1]$$

In [ ]:
def precision(output: torch.Tensor, target: torch.Tensor, topk: int) -> torch.Tensor:
    '''
    output, target ~ (users, items)
    target_sorted_by_output ~ (users, items)
    '''
    target_sorted_by_output = prepare_target(output, target)
    # YOUR CODE HERE
    return 0

In [ ]:
tests.run_precision(precision)

# Recall

$$R@k = \frac{\sum_{i=1}^k [y_{i} = 1]}{\sum_{i=1}^N [y_{i} = 1]}$$

In [ ]:
def recall(output: torch.Tensor, target: torch.Tensor, topk: int) -> torch.Tensor:
        '''
    output, target ~ (users, items)
    target_sorted_by_output ~ (users, items)
    '''
    target_sorted_by_output = prepare_target(output, target)
    # YOUR CODE HERE
    return 0

In [ ]:
tests.run_recall(recall)

# Mean (Normalized) Average Precision


$$AP@k = \sum_{i=1}^{k} \frac{y_{i}}{\sum_{j=1}^{k} \cdot y_{j}} p@i$$
$$MNAP@k = \frac{1}{U} \sum_{u \in U} \frac{1}{min(k, n_u)} \sum_{i=1}^{k}y_{i}P@k_{i},$$

где $n_u$ - количество айтемов с интеракциями у пользователя $u$ в тестовый период, <br>
$U$ - количество пользователей. 

In [ ]:
def mnap(output: torch.Tensor, target: torch.Tensor, topk: int, normalized: bool = True) -> torch.Tensor:
    '''
    output, target ~ (users, items)
    target_sorted_by_output ~ (users, items)
    '''
    target_sorted_by_output = prepare_target(output, target)
    # YOUR CODE HERE
    return 0

In [ ]:
tests.run_map(mnap)
tests.run_mnap(mnap)

# Normalized Dicsounted Cumulative Gain


$$ NDCG @k = \frac{DCG@k}{IDCG@k},$$ где 
$$DCG@k = \sum_{i=1}^{k} \frac{2^{y_{i}} - 1}{log_2 (i + 1)}$$
$$IDCG@k = \sum_{i=1}^{k} \frac{1}{log_2 (i + 1)}$$

In [ ]:
def dcg(tensor: torch.Tensor) -> torch.Tensor:
    gains = (2**tensor) - 1
    return gains / torch.log2(torch.arange(0, tensor.size(-1), dtype=torch.float, device=tensor.device) + 2.0)


def ndcg(output: torch.Tensor, target: torch.Tensor, topk: int) -> torch.Tensor:
    '''
    output, target ~ (users, items)
    target_sorted_by_output ~ (users, items)
    '''
    target_sorted_by_output = prepare_target(output, target)
    ideal_target = prepare_target(target, target)
    # YOUR CODE HERE
    return 0

In [ ]:
tests.run_ndcg(ndcg)